#  Contents 
##  : Function to import data using postgress 
        - SQL_Data(postgres_user = '', postgres_pw = '',\
             postgres_host = '', postgres_port = '',\
             postgres_db ='')
##  : Checks for normality in a vector
        - NormalityCheck(data, bins = 25, alpha=0.05)
##  : Checks For Gauss-Markov Conditions of OLS from with inputs as the Design Matrix and the Target Variable 
        - GausMarkovCheck(Y, X, alpha= 0.05, bins = 25)
## : Plot the correlation heatmap
        - cor_map(data)


In [1]:
#! pip install ipynb
#! pip install nbloader

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
# Function to import data using postgress 

from sqlalchemy import create_engine
import warnings
import pandas as pd
    
def SQL_Data(postgres_user = '', postgres_pw = '',\
             postgres_host = '', postgres_port = '',\
             postgres_db =''):  

    warnings.filterwarnings('ignore')
    variable = postgres_db
    engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
    
    data = pd.read_sql_query(str('select * from '+ variable),con=engine)
    engine.dispose()
    return(data)

# Checks

In [3]:
# Checks for normality in a vector

# Data: must be a vector we are interested checking
# Bins: Number of bins in a 

import statsmodels.api as sm
import scipy.stats as stats
from scipy.stats import shapiro
from scipy.stats import normaltest
from scipy.stats import anderson
from scipy.stats import jarque_bera

def NormalityCheck(data, bins = 25, alpha=0.05):  
    fig, ax = plt.subplots(figsize=(18,5), nrows=1, ncols=2)
    sns.distplot(data, ax=ax[0], bins=bins)
    sm.qqplot(data, stats.norm, ax=ax[1])
    plt.show()
    print("")
    stat, p = shapiro(data)          # test
        # interpret
    alpha = alpha
    if p > alpha:
        print('According to Shapiro, the sample looks Gaussian (fail to reject H0)')
    else:
        print('According to Shapiro, the sample does not look Gaussian (reject H0)')
    print('Statistics=%.3f, p=%.3f' % (stat, p))
    
    print("")
    stat, p = normaltest(data)          # test
    # interpret
    alpha = alpha
    if p > alpha:
        print('According to D’Agostino’s K^2 Normality Test, the sample looks Gaussian (fail to reject H0)')
    else:
        print('According to D’Agostino’s K^2 Normality Test, the sample does not look Gaussian (reject H0)')
    print('Statistics=%.3f, p=%.3f' % (stat, p))             
    
    print("")
    stat, p = jarque_bera(data)          # test
    # interpret
    alpha = alpha
    if p > alpha:
        print('According to Jarque Bera, the sample looks Gaussian (fail to reject H0)')
    else:
        print('According to Jarque Bera, the sample does not look Gaussian (reject H0)')
    print('Statistics=%.3f, p=%.3f' % (stat, p))

In [4]:
#Checks For Gauss-Markov Conditions of OLS from with inputs as the Design Matrix and the Target Variable 
# Function automatically adds constant

import statsmodels.api as sm
import scipy
from scipy.stats import bartlett
from scipy.stats import levene
from statsmodels.tsa.stattools import acf
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

## X: Design Matrix; Ensure no missing variables to avoid problems
## Y: Target Variable vector
## alpha = Significance level of tests
## number of bins in histograms

def GausMarkovCheck(Y, X, alpha= 0.05, bins = 25):
    warnings.filterwarnings('ignore')
    results = sm.OLS(Y, sm.add_constant(X)).fit()
    print(results.summary())

    pred_val = results.fittedvalues.copy()
    true_val = Y.values.copy()
    residual = true_val.ravel() - pred_val.ravel()

    print ('')
    print ('1) Check Linearity in the coefficients')
    if (X.shape[1] % 2) == 0:
        nrows = X.shape[1] // 2
    else:
        nrows = (X.shape[1] // 2) + 1
    fig, ax = plt.subplots(figsize=(18,(5*nrows)), ncols=2, nrows=nrows)
    i = 0
    j = 0
    for item in list(X.columns):
        if nrows != 1:
            sns.scatterplot(x=X[item], y=residual, ax=ax[i][j])
            ax[i][j].set_ylabel("residual")
        else:
            sns.scatterplot(x=X[item], y=residual, ax=ax[j])
            ax[j].set_ylabel("residual")
            
        if j < 1 :
            j += 1
        else:
            i += 1
            j = 0
    plt.show()
    
    print ('')
    print ('2) Check Error term should be 0')
    print ('')
    stat, p = scipy.stats.ttest_1samp(residual, 0)          # test
        # interpret
    alpha = alpha
    if p > alpha:
        print('T-tests suggests the residual mean is 0 (fail to reject H0)')
    else:
        print('T-tests suggests the residual mean is not 0 (reject H0)')
    print('Statistics=%.3f, p=%.3f' % (stat, p))
 
    print ('')
    print ('3) Check for Homoscedacity')
    fig, ax = plt.subplots(figsize=(18, 5), nrows=1, ncols=1)
    sns.scatterplot(x=pred_val, y=residual)
    plt.title('Residual vs Fit')
    plt.show()
    
    print ('')
    stat, p = bartlett(pred_val, residual)          # test
    # interpret
    if p > alpha:
        print('According to Bartlett\'s Test, the sample looks Homoscedastic (fail to reject H0)')
    else:
        print('According to Bartlett\'s Test, the sample does not look Homoscedastic (reject H0)')
    print('Statistics=%.3f, p=%.3f' % (stat, p))
    
    print("")
    stat, p = levene(pred_val, residual)          # test
    # interpret
    if p > alpha:
        print('According to Levene\'s Test, the sample looks Homoscedastic (fail to reject H0)')
    else:
        print('According to Levene\'s Test, the sample does not look Homoscedastic (reject H0)')
    print('Statistics=%.3f, p=%.3f' % (stat, p))  
    
    print ('')
    print ('4) Check for Uncorrolated Error terms')
    acf_data = acf(residual)
    fig, ax = plt.subplots(figsize=(18,5), ncols=2, nrows=1)
    ax[0].plot(residual)
    ax[0].set_title('Line Plot of Residuals')
    ax[1].plot(acf_data[1:])
    ax[1].set_title('Autocorrelation Plot of Residuals')
    plt.show()
    
    print ('')
    print ('5 & 6) Check for Multicolinearity and Feature correlation with error')
    
    print ('')
    print('Variance Inflation Factors')
    print(pd.Series([variance_inflation_factor(add_constant(X.select_dtypes([np.number]).dropna()).values, i) \
                    for i in range(add_constant(X.select_dtypes([np.number]).dropna()).shape[1])],\
                    index=(add_constant(X.select_dtypes([np.number]).dropna()).columns)))
    
    cor_map(pd.concat([X, pd.DataFrame(residual).rename(columns={0: 'residuals'})], axis=1))
    
    print ('')
    print ('7) Check for Normality of the Error')    
    NormalityCheck(residual, bins = bins, alpha=alpha)
    

In [5]:
# Plot the correlation heatmap

import matplotlib.pyplot as plt
import seaborn as sns

## Data: The data set we are interested in checking  the correlation for

def cor_map(data): 
    fig, ax = plt.subplots(figsize=(13,13), nrows=1, ncols=1)
    corrmat = data.corr()
    sns.heatmap(corrmat, center=0, square=True, annot=True, linewidths=.5)
    plt.title('correlation matrix')
    plt.show()

In [6]:
## Get transformed data from PCA

from sklearn.decomposition import PCA

def PCA_Trans(data, n):
    pca = PCA(n_components=n)   #if we want to to select number of components , can also look at % variance saved if 0< n_components < 1
    pca.fit(data.dropna())
    pca_expenditure = pca.transform(data.dropna())
    return(pd.DataFrame(pca_expenditure))

# Visuals

In [7]:
## histogram of all data
def All_Hist(data, bins, norm_hist=False):
    if (data.select_dtypes([np.number]).shape[1] % 3) == 0:
        nrows = data.select_dtypes([np.number]).shape[1] // 3
    else:
        nrows = (data.select_dtypes([np.number]).shape[1] // 3) + 1
    fig, ax = plt.subplots(figsize=(18,(5*nrows)), ncols=3, nrows=nrows)
    i = 0
    j = 0
    for item in list(data.select_dtypes([np.number]).columns):
        if nrows != 1:
            sns.distplot(data.select_dtypes([np.number])[item], ax=ax[i][j], bins=bins, norm_hist=norm_hist)
            ax[i][j].set_ylabel(item)
        else:
            sns.distplot(data.select_dtypes([np.number])[item], ax=ax[j], bins=bins, norm_hist=norm_hist)
            ax[j].set_ylabel("residual")
        if j < 2 :
            j += 1
        else:
            i += 1
            j = 0
    plt.show()

In [8]:
## scatter plot of Y vs all data columns
# Y= Target variable
#data=dataset
# hue= hue, categorical variable
def All_Scatter(Y, data, hue=None):
    if (data.drop([Y], axis=1).select_dtypes([np.number]).shape[1] % 3) == 0:
        nrows = data.select_dtypes([np.number]).shape[1] // 3
    else:
        nrows = (data.drop([Y], axis=1).select_dtypes([np.number]).shape[1] // 3) + 1
    fig, ax = plt.subplots(figsize=(18,(5*nrows)), ncols=3, nrows=nrows)
    i = 0
    j = 0
    for item in list(data.drop([Y], axis=1).select_dtypes([np.number]).columns):
        if nrows != 1:
            sns.scatterplot(y=Y, x=item, data=data, ax=ax[i][j], hue=hue)
            ax[i][j].set_ylabel(Y)
        else:
            sns.scatterplot(y=Y, x=item, data=data, ax=ax[j], hue=hue)
            ax[j].set_ylabel(Y)
        if j < 2 :
            j += 1
        else:
            i += 1
            j = 0
    plt.show()
    
# if problems, remove hue from each part

In [9]:
#boxplots of all categoricals/objects

def All_Boxen(Y, data):
    if (data.drop([Y], axis=1).select_dtypes([np.object]).shape[1] % 3) == 0:
        nrows = data.select_dtypes([np.object]).shape[1] // 3
    else:
        nrows = (data.drop([Y], axis=1).select_dtypes([np.object]).shape[1] // 3) + 1
    fig, ax = plt.subplots(figsize=(18,(5*nrows)), ncols=3, nrows=nrows)
    i = 0
    j = 0
    for item in list(data.drop([Y], axis=1).select_dtypes([np.object]).columns):
        if nrows != 1:
            sns.boxenplot(y=Y, x=item, data=data, ax=ax[i][j])
            ax[i][j].set_ylabel(Y)
        else:
            sns.boxenplot(y=Y, x=item, data=data, ax=ax[j])
            ax[j].set_ylabel(Y)
        if j < 2 :
            j += 1
        else:
            i += 1
            j = 0
    plt.show()

## Variable Selection

In [10]:
#See number and percentage of variables that have missing values 

def Missing_Var(data):
    return(pd.concat([data.isna().sum().rename('# Missing'), 
    ((data.isna().sum())/data.shape[0]).rename('% Missing')], axis = 1 )[list(data.isna().sum()>0)])

In [11]:
# get Variance Inflation Factors

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

# automatically drops NA and inserts constant
def VIF(X):
    print(pd.Series([variance_inflation_factor(add_constant(X.select_dtypes([np.number]).dropna()).values, i) \
                    for i in range(add_constant(X.select_dtypes([np.number]).dropna()).shape[1])],\
                    index=(add_constant(X.select_dtypes([np.number]).dropna()).columns)))

In [13]:
# feature selection by VIF
# data = design matrix
# cutoff

def VIF_selection(data, cutoff=5):
    Cdata=data   #clean data
    X= add_constant(Cdata.select_dtypes([np.number]).dropna())
    VarFac=pd.Series([variance_inflation_factor(X.values, i) for i in range(X.shape[1])], index=(X.columns))    
    while max(VarFac[VarFac.index.values != 'const']) > cutoff:
        X= add_constant(Cdata.select_dtypes([np.number]).dropna())
        VarFac=pd.Series([variance_inflation_factor(X.values, i) for i in range(X.shape[1])], index=(X.columns))
        Cdata=Cdata.drop([np.argmax(VarFac[VarFac.index.values != 'const'])], axis = 1)
    return(Cdata, VIF(Cdata))

# Cdata =  selected data set

# Cross Validation

In [1]:
# Split a list into random groups
    # done by list generation, not for loops

#a: list
#n: number of sets

def split(a, n):
    k, m = divmod(len(a), n)
    return (a[i * k + min(i, m):(i + 1) * k + min(i + 1, m)] for i in range(n))

# notes:
#k: numerator of len(a)/n
#m: remainder of len(a)/n 
    
# notes:
# if remainder after division is shorter of greater will decide final list
# The real problem is looking ate the final list
#
# credit
# https://stackoverflow.com/questions/3352737/python-randomly-partition-a-list-into-n-nearly-equal-parts

In [ ]:
#Takes in data and cross validates 

from random import shuffle
from sklearn.naive_bayes import BernoulliNB

#Target:Target colun
#Data:Design matrix with target attatched as colummn
#n: nuumber of cross validation

def Cross_Val(data, n=5, Target='Positive'):
    S_Index = list(range(data.shape[0]))             # get observation indices
    shuffle(S_Index)                               #shuffle the data points
    k, m = divmod(len(S_Index), n)
    g=[S_Index[i * k + min(i, m):(i + 1) * k + min(i + 1, m)] for i in range(n)]    # g: list of divided lists, division
    for item in range(n):
        g_test=g[item]
        g_train=g[:item]+g[item+1:]
        g_train = [item for sublist in g_train for item in sublist]
        
        bnb = BernoulliNB()         # bernouli Naive Bays
        # Fit our model to the data.
        bnb.fit((data.iloc[g_train]).drop(columns=[Target]), data.iloc[g_train][Target])
        
        # Classify, storing the result in a new variable.
        y_pred = bnb.predict((data.iloc[g_test]).drop(columns=[Target]))
        print((((data.iloc[g_test])[Target] != y_pred).sum())/len(y_pred))